In [1]:
from datasets import load_dataset
dataset = load_dataset("ag_news")

README.md: 0.00B [00:00, ?B/s]

e:\AnacondaPython\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saura\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [3]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [20]:
from transformers import AutoTokenizer

In [21]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

e:\AnacondaPython\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saura\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
def tokenize_for_mlm(txt):
    return tokenizer(
        txt["text"],
        truncation = True,
        max_length=128,
        padding="max_length"
    )

In [23]:
mlm_dataset = dataset['train'].map(tokenize_for_mlm, batched=True, remove_columns=['label'])

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

In [24]:
mlm_dataset.set_format(type="torch", columns=['input_ids','attention_mask'])

In [25]:
from transformers import DataCollatorForLanguageModeling

In [26]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15) #Mask 15% of tokens dynamic masking at runtime

In [27]:
sample = mlm_dataset[0]
collated_sample = data_collator([sample])
tokenizer.convert_ids_to_tokens(collated_sample["input_ids"][0][:10])

['[CLS]',
 '[MASK]',
 'st',
 '[MASK]',
 'bears',
 '[MASK]',
 'back',
 'into',
 'the',
 'black']

In [28]:
from transformers import AutoModelForMaskedLM

In [29]:
mlm_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-dapt-news",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True
)

In [31]:
from transformers import Trainer

trainer = Trainer(
    model=mlm_model,
    args=training_args,
    train_dataset=mlm_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

C:\Users\saura\AppData\Local\Temp\ipykernel_9668\2221508341.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,2.714400
200,2.469000
300,2.342700
400,2.344800
500,2.279900
600,2.233300
700,2.161900
800,2.163100
900,2.176200
1000,2.150600


TrainOutput(global_step=15000, training_loss=1.8550681518554688, metrics={'train_runtime': 58208.7951, 'train_samples_per_second': 4.123, 'train_steps_per_second': 0.258, 'total_flos': 1.5792288768e+16, 'train_loss': 1.8550681518554688, 'epoch': 2.0})

In [33]:
trainer.save_model("./bert-dapt-news")

In [34]:
tokenizer.save_pretrained("./bert-dapt-news")

('./bert-dapt-news\\tokenizer_config.json',
 './bert-dapt-news\\special_tokens_map.json',
 './bert-dapt-news\\vocab.txt',
 './bert-dapt-news\\added_tokens.json',
 './bert-dapt-news\\tokenizer.json')

In [35]:
def tokenize_for_classification(txt):
    return tokenizer(
        txt['text'],
        truncation = True,
        padding="max_length",
        max_length=128
    )

In [36]:
encoded_dataset = dataset.map(tokenize_for_classification, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [37]:
encoded_dataset.set_format(type="torch", columns=["input_ids","attention_mask","label"])

In [40]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [41]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "Accuracy": accuracy_score(labels, preds),
        "F1 Score": f1_score(labels, preds, average="weighted")
    }

In [45]:
finetune_args = TrainingArguments(
    output_dir="./bert-agnews",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
)

In [46]:
from transformers import AutoModelForSequenceClassification

In [48]:
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

trainer_base = Trainer(
    model = base_model,
    args=finetune_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer_base.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\saura\AppData\Local\Temp\ipykernel_9668\1369776359.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_base = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 score
1,0.193600,0.179496,0.942237,0.942214
2,0.124000,0.190127,0.948026,0.948083
3,0.080000,0.238636,0.947237,0.947272


TrainOutput(global_step=22500, training_loss=0.14923132752312554, metrics={'train_runtime': 82082.6684, 'train_samples_per_second': 4.386, 'train_steps_per_second': 0.274, 'total_flos': 2.368042020864e+16, 'train_loss': 0.14923132752312554, 'epoch': 3.0})

In [49]:
base_metrics = trainer_base.evaluate()
base_metrics

{'eval_loss': 0.17949649691581726,
 'eval_Accuracy': 0.9422368421052632,
 'eval_F1 Score': 0.9422144586416876,
 'eval_runtime': 551.7008,
 'eval_samples_per_second': 13.776,
 'eval_steps_per_second': 0.861,
 'epoch': 3.0}

In [50]:
dapt_model = AutoModelForSequenceClassification.from_pretrained("./bert-dapt-news", num_labels = 4)
trainer_dapt = Trainer(
    model=dapt_model,
    args=finetune_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer_dapt.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert-dapt-news and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\saura\AppData\Local\Temp\ipykernel_9668\548927377.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_dapt = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 score
1,0.190800,0.175723,0.944605,0.944746
2,0.113500,0.187510,0.948421,0.948532
3,0.075100,0.232574,0.947500,0.947550


TrainOutput(global_step=22500, training_loss=0.1426836507373386, metrics={'train_runtime': 135621.6716, 'train_samples_per_second': 2.654, 'train_steps_per_second': 0.166, 'total_flos': 2.368042020864e+16, 'train_loss': 0.1426836507373386, 'epoch': 3.0})

In [51]:
dapt_metrics = trainer_dapt.evaluate()
dapt_metrics

{'eval_loss': 0.17572283744812012,
 'eval_Accuracy': 0.9446052631578947,
 'eval_F1 Score': 0.9447463206403587,
 'eval_runtime': 636.7329,
 'eval_samples_per_second': 11.936,
 'eval_steps_per_second': 0.746,
 'epoch': 3.0}

DAPT (Domain Adaptive Pretraining) + BERT using MLM (Masked Language Modelling) increased the accuracy by 0.25% compared to BERT